# Crawl match instances

In [147]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

from_year = 2017
to_year = 2018

driver = webdriver.Chrome()
#driver = webdriver.Safari()
if (from_year == 2016 and to_year == 2017):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
elif (from_year == 2017 and to_year == 2018):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/Fixtures/Italy-Serie-A-2017-2018"

    driver.get(start_url)

In [148]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")
        
    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_status = tds[2].find_element_by_tag_name("span").get_attribute("textContent")
                    if (match_status != "FT"):
                        continue
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

wait


In [153]:
# Crawl data of each match details
print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in tqdm(match_table.items()):
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    subdriver.get(match_link)
    time.sleep(1)
    web_match_trs = subdriver.find_elements_by_xpath("//*[@id=\"match-header\"]/table/tbody/tr")

    # Crawl basic match information
    web_match_tr = web_match_trs[0]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    home_team_id = int(re.findall(r'\d+', web_match_tds[0].find_element_by_tag_name("a").get_attribute("href"))[0])
    away_team_id = int(re.findall(r'\d+', web_match_tds[2].find_element_by_tag_name("a").get_attribute("href"))[0])

    web_match_tr = web_match_trs[1]
    web_match_tds = web_match_tr.find_elements_by_tag_name("td")
    web_match_divs = web_match_tds[1].find_elements_by_tag_name("div")
    match_datetime = web_match_divs[2].find_element_by_xpath("dl/dd[2]").text + ", " + web_match_divs[2].find_element_by_xpath("dl/dd[1]").text
    half_time_result = web_match_divs[1].find_element_by_xpath("dl/dd[1]").text
    result = web_match_divs[1].find_element_by_xpath("dl/dd[2]").text
    
    match_table[match_id]["match_datetime"] = match_datetime
    match_table[match_id]["home_team_id"] = home_team_id
    match_table[match_id]["away_team_id"] = away_team_id
    match_table[match_id]["half_time_result"] = half_time_result
    match_table[match_id]["result"] = result
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:273.
>>> 1: Crawling 1225358...
>>> 2: Crawling 1225233...
>>> 3: Crawling 1225444...
>>> 4: Crawling 1225392...
>>> 5: Crawling 1225247...
>>> 6: Crawling 1225338...
>>> 7: Crawling 1225371...
>>> 8: Crawling 1225337...
>>> 9: Crawling 1225223...
>>> 10: Crawling 1225339...
>>> 11: Crawling 1225222...
>>> 12: Crawling 1225397...
>>> 13: Crawling 1225162...
>>> 14: Crawling 1225366...
>>> 15: Crawling 1225218...
>>> 16: Crawling 1225201...
>>> 17: Crawling 1225334...
>>> 18: Crawling 1225393...
>>> 19: Crawling 1225285...
>>> 20: Crawling 1225309...
>>> 21: Crawling 1225297...
>>> 22: Crawling 1225390...
>>> 23: Crawling 1225272...
>>> 24: Crawling 1225231...
>>> 25: Crawling 1225262...
>>> 26: Crawling 1225260...
>>> 27: Crawling 1225188...
>>> 28: Crawling 1225329...
>>> 29: Crawling 1225380...
>>> 30: Crawling 1225313...
>>> 31: Crawling 1225373...
>>> 32: Crawling 1225229...
>>> 33: Crawling 1225182...
>>> 34: Crawling 1225195...
>>> 35: Crawling 1225306...


In [154]:
import json

with open('../Data/{}-{}/match_instances_table.json'.format(from_year,to_year), 'w') as fp:
    json.dump(match_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [37]:
import pandas as pd
tmp = pd.read_json('../Data/{}-{}/match_instances_table.json'.format(from_year,to_year), orient='index')

# Crawl team instances

In [155]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

from_year = 2017
to_year = 2018

driver = webdriver.Chrome()
if (from_year == 2016 and to_year == 2017):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/TeamStatistics/Italy-Serie-A-2016-2017"
elif (from_year == 2017 and to_year == 2018):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/TeamStatistics/Italy-Serie-A-2017-2018"

driver.get(start_url)

In [156]:
team_table = {}

trs = driver.find_elements_by_xpath("//*[@id=\"top-team-stats-summary-content\"]/tr")

for tr in trs:
    team_link = tr.find_element_by_class_name("team-link").get_attribute("href")
    team_id = int(re.findall(r'\d+',team_link)[0])
    team_table[team_id] = {}
    team_table[team_id]["team_link"] = team_link

In [157]:
# Crawl data of each team
print("Total number of teams:{}.".format(len(team_table)))
cnt = 1
subdriver = webdriver.Chrome()
for team_id, team_details in tqdm(team_table.items()):
    team_link = team_details['team_link']
    # Change into the team page
    print(">>> {}: Crawling {}...".format(cnt, team_id))
    
    while True:
        try:
            subdriver.get(team_link)
            time.sleep(1)

            web_team_img_link = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/img").get_attribute("src")
            web_team_name = subdriver.find_element_by_xpath("//*[@id=\"layout-content-wrapper\"]/div[2]/div[1]/h2/span").text
            break
        except:
            time.sleep(1)
    
    team_table[team_id]["team_img_link"] = web_team_img_link
    team_table[team_id]["team_name"] = web_team_name
    
    cnt += 1
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of teams:20.
>>> 1: Crawling 779...
>>> 2: Crawling 2889...
>>> 3: Crawling 278...
>>> 4: Crawling 71...
>>> 5: Crawling 72...
>>> 6: Crawling 73...
>>> 7: Crawling 75...
>>> 8: Crawling 300...
>>> 9: Crawling 77...
>>> 10: Crawling 78...
>>> 11: Crawling 271...
>>> 12: Crawling 80...
>>> 13: Crawling 76...
>>> 14: Crawling 84...
>>> 15: Crawling 86...
>>> 16: Crawling 87...
>>> 17: Crawling 267...
>>> 18: Crawling 276...
>>> 19: Crawling 1502...
>>> 20: Crawling 1503...
>>> finished.


In [158]:
import json

with open('../Data/{}-{}/team_instances_table.json'.format(from_year,to_year), 'w') as fp:
    json.dump(team_table, fp, indent=4)

### Example for reading JSON as Dataframe

In [68]:
import pandas as pd
tmp = pd.read_json('../Data/{}-{}/team_instances_table.json'.format(from_year,to_year), orient='index')

# Crawl player instances

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

from_year = 2017
to_year = 2018

driver = webdriver.Chrome()

if (from_year == 2016 and to_year == 2017):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/PlayerStatistics/Italy-Serie-A-2016-2017"
elif (from_year == 2017 and to_year == 2018):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/PlayerStatistics/Italy-Serie-A-2017-2018"
driver.get(start_url)

ALL_PLAYER = True

In [3]:
player_table = {}

if ALL_PLAYER:
    all_players_button = driver.find_element_by_xpath("//*[@id=\"apps\"]/dd[2]/a")
    all_players_button.click()

time.sleep(1)

In [4]:
next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
next_button_status = next_button.get_attribute("class")

player_cnt = 0
player_id_list = []
cnt = 1

ttl_players = 0

while next_button_status != "option  disabled ":
    print(">>> Crawling page {}...".format(cnt))
    while True:
        try:
            player_statistic_tbody = driver.find_element_by_xpath("//*[@id=\"player-table-statistics-body\"]")
            player_trs = player_statistic_tbody.find_elements_by_xpath("tr")
            ttl_players += len(player_trs)
            for tr in player_trs:
                pn = tr.find_element_by_class_name("pn")
                player_link = pn.find_element_by_class_name("player-link").get_attribute("href")
                player_id = int(re.findall(r'\d+', player_link)[0])

                player_table[player_id] = {}
                player_table[player_id]["player_link"] = player_link

                player_id_list.append(player_id)
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
    next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    while True:
        try:
            next_button.click()
            break
        except:
            print(">>> error...waiting next click...")
            time.sleep(1)
            next_button = driver.find_element_by_xpath("//*[@id=\"next\"]")
    cnt += 1
    next_button_status = next_button.get_attribute("class")
    time.sleep(2)

print(">>> finished.")

>>> Crawling page 1...
>>> Crawling page 2...
>>> Crawling page 3...
>>> Crawling page 4...
>>> Crawling page 5...
>>> Crawling page 6...
>>> Crawling page 7...
>>> Crawling page 8...
>>> Crawling page 9...
>>> Crawling page 10...
>>> Crawling page 11...
>>> Crawling page 12...
>>> Crawling page 13...
>>> Crawling page 14...
>>> Crawling page 15...
>>> Crawling page 16...
>>> Crawling page 17...
>>> Crawling page 18...
>>> Crawling page 19...
>>> Crawling page 20...
>>> Crawling page 21...
>>> error...waiting next trial...
>>> Crawling page 22...
>>> Crawling page 23...
>>> Crawling page 24...
>>> Crawling page 25...
>>> Crawling page 26...
>>> Crawling page 27...
>>> Crawling page 28...
>>> Crawling page 29...
>>> Crawling page 30...
>>> Crawling page 31...
>>> Crawling page 32...
>>> Crawling page 33...
>>> Crawling page 34...
>>> Crawling page 35...
>>> Crawling page 36...
>>> Crawling page 37...
>>> Crawling page 38...
>>> Crawling page 39...
>>> Crawling page 40...
>>> Crawling pa

In [44]:
# Crawl data of each player
print("Total number of players:{}.".format(len(player_table)))
cnt = 1
subdriver = webdriver.Chrome()
for player_id, player_details in tqdm(player_table.items()):
    player_link = player_details['player_link']
    if ("team_id" in player_details):
        if (player_details["team_id"] is not None):
            continue

    while True:
        try:
            # Change into the player page
            print(">>> {}: Crawling {}...".format(cnt, player_id))
            subdriver.get(player_link)
            time.sleep(1)

            team_id = None
            web_player_name = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[1]/dd").text
            web_player_img_link = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/img").get_attribute("src")
            web_player_birthday = subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[1]/dd/i").text        
            try:
                web_player_height = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[2]/dd").text)[0])
            except:
                web_player_height = "N/A"
            try:
                web_player_weight = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[2]/dl[3]/dd").text)[0])
            except:
                web_player_weight = "N/A"
            no_dls = subdriver.find_elements_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl")
            web_player_position_items = no_dls[-1].find_elements_by_xpath("dd/ul/li")
            web_player_position = [item.text for item in web_player_position_items]
            
            history_link = subdriver.find_element_by_xpath("//*[@id=\"sub-navigation\"]/ul/li[4]/a").get_attribute("href")
            
            while True:
                try:
                    # Change into the history page
                    subdriver.get(history_link)
                    time.sleep(1)

                    player_table_trs = subdriver.find_elements_by_xpath("//*[@id=\"player-table-statistics-body\"]/tr")
                    if (len(player_table_trs) == 0):
                        raise Exception("History not loaded.")

                    for tr in player_table_trs[:-1]:
                        season_year = None
                        league = None
                        while True:
                            try:
                                season_year = tr.find_element_by_xpath("td[1]").text
                                league = tr.find_element_by_xpath("td[3]/a").text
                                print("season_year: {}".format(season_year))

                                if season_year == ("{}/{}".format(from_year,to_year)) and league == "ISA":
                                    team_id = int(re.findall(r'\d+', tr.find_element_by_class_name("team-link").get_attribute("href"))[0])
                                break
                            except Exception as e:
                                print(e)
                                time.sleep(1)
                    break
                except Exception as e:
                    print(e)
                    time.sleep(1)
                
            # team_id = int(re.findall(r'\d+', subdriver.find_element_by_xpath("//*[@id=\"player-profile\"]/div/div/div[1]/dl[2]/dd/a").get_attribute("href"))[0])
            
            if (team_id is None):
                raise Exception("team_id is None, try again...")
            
            player_table[player_id]["player_name"] = web_player_name   
            player_table[player_id]["player_img_link"] = web_player_img_link
            player_table[player_id]["player_birthday"] = web_player_birthday
            player_table[player_id]["player_height"] = web_player_height
            player_table[player_id]["player_weight"] = web_player_weight
            player_table[player_id]["team_id"] = team_id
            player_table[player_id]["player_position"] = web_player_position
            
            break
        except Exception as e:
            print(e)
            print(">>> error...waiting next trial...")
            time.sleep(1)
        
    cnt += 1

print(">>> finished")
driver.close()
subdriver.close()   

Total number of players:560.


A Jupyter Widget


>>> finished


ConnectionRefusedError: [Errno 61] Connection refused

In [42]:
player_table[103383]

{'player_link': 'https://www.whoscored.com/Players/103383/Show/Gustavo-G%C3%B3mez'}

In [45]:
import json

with open('../Data/{}-{}/player_instances_table.json'.format(from_year,to_year), 'w') as fp:
    json.dump(player_table, fp, indent=4)

# Crawl performance instances

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import re
import time
from tqdm import tqdm_notebook as tqdm

from_year = 2016
to_year = 2017

driver = webdriver.Chrome()
#driver = webdriver.Safari()

if (from_year == 2016 and to_year == 2017):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6461/Stages/14014/Fixtures/Italy-Serie-A-2016-2017"
elif (from_year == 2017 and to_year == 2018):
    start_url = "https://www.whoscored.com/Regions/108/Tournaments/5/Seasons/6974/Stages/15404/Fixtures/Italy-Serie-A-2017-2018"

driver.get(start_url)

In [2]:
# Find the link of each match
match_table = {}

prev_month_button = driver.find_element_by_xpath("//*[@id=\"date-controller\"]/a[1]")
prev_button_status = prev_month_button.get_attribute("title")
previous_month = ""
current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text

while (prev_button_status == "View previous month"):
    while (current_month == previous_month):
        prev_month_button.click()
        time.sleep(1)
        current_month = driver.find_element_by_xpath("//*[@id=\"date-config-toggle-button\"]/span[1]").text
        prev_button_status = prev_month_button.get_attribute("title")

    while True:
        try:
            # match_table = {}
            trs = driver.find_elements_by_xpath("//*[@id=\"tournament-fixture\"]/tbody/tr")

            for tr in trs:
                tds = tr.find_elements_by_xpath("td")
                if(len(tds)):
                    match_status = tds[2].find_element_by_tag_name("span").get_attribute("textContent")
                    if (match_status != "FT"):
                        continue
                    match_id = tr.get_attribute("data-id")
                    match_table[match_id] = {}
                for i, td in enumerate(tds):
                    if(i == 4):
                        match_table[match_id]["match_link"] = td.find_element_by_tag_name("a").get_attribute("href")
            break
        except:
            print("wait")
            time.sleep(0.5)

    previous_month = current_month

In [ ]:
# Crawl data of each match performance
performance_table = {}

print("Total number of matches:{}.".format(len(match_table)))
cnt = 1
subdriver = webdriver.Chrome()
for match_id, match_details in tqdm(match_table.items()):
    performance_table[match_id] = {}
    match_link = match_details["match_link"]
    # Change into the match page
    print(">>> {}: Crawling {}...".format(cnt, match_id))
    cnt += 1
    
    while True:
        try:
            subdriver.get(match_link)
            time.sleep(1)
            
            # Record the match time first
            match_time = subdriver.find_element_by_xpath("//*[@id=\"match-centre-timeline\"]/div[1]/div[2]/div[2]/div[2]/div[2]/span").get_attribute("textContent")
            match_time = sum([int(digit) for digit in match_time.split("+")])
            
            # Change into player statistic page
            subdriver.get(subdriver.find_element_by_xpath("//*[@id=\"sub-sub-navigation\"]/ul/li[2]/a").get_attribute("href"))
            time.sleep(1)
            
            # Click through all player statistic tables such that they can be loaded
            # offensive button
            home_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[2]/a")
            home_offensive_button.click()
            time.sleep(2)
            away_offensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[2]/a")
            away_offensive_button.click()
            time.sleep(2)

            # defensive button
            home_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[3]/a")
            home_defensive_button.click()
            time.sleep(2)
            away_defensive_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[3]/a")
            away_defensive_button.click()
            time.sleep(2)
            
            # Passing button
            home_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-home-options\"]/li[4]/a")
            home_passing_button.click()
            time.sleep(2)
            away_passing_button = subdriver.find_element_by_xpath("//*[@id=\"live-player-away-options\"]/li[4]/a")
            away_passing_button.click()
            time.sleep(2)
            
            # Crawl match performance
            #home_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_offensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-offensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_offensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-offensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_offensive_trs = home_offensive_trs + away_offensive_trs

            for tr in total_offensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])
                performance_table[match_id][player_id] = {}

                player_position = tr.find_element_by_xpath("td[3]/span[2]").get_attribute("textContent")
                player_position = ''.join(filter(str.isalpha, player_position))
                
                #total_shots = tr.find_element_by_class_name("ShotsTotal ").text
                total_shots = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #ot_shots = tr.find_element_by_class_name("ShotOnTarget ").text
                ot_shots = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #key_passes = tr.find_element_by_class_name("KeyPassTotal ").text
                key_passes = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #dribbles = tr.find_element_by_class_name("DribbleWon ").text
                dribbles = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #foul_given = tr.find_element_by_class_name("FoulGiven ").text
                foul_given = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #offsides = tr.find_element_by_class_name("OffsideGiven ").text
                offsides = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #disp = tr.find_element_by_class_name("Dispossessed ").text
                disp = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #uns_touches = tr.find_element_by_class_name("Turnover ").text
                uns_touches = tr.find_element_by_xpath("td[11]").get_attribute("textContent")

                rating = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                key_events_icons = tr.find_elements_by_class_name("incident-icon")
                key_events = ["{},{},{}".format(icon.get_attribute("data-type"), icon.get_attribute("data-minute"), icon.get_attribute("data-second")) for icon in key_events_icons]

                if (rating == "-"):
                    minutes_played = 0
                else:
                    minutes_played = match_time

                for event in key_events:
                    event_type = event.split(",")[0]
                    event_minute = event.split(",")[1]
                    if (event_type == "18"):
                        minutes_played = int(event_minute)
                        break
                    elif (event_type == "19"):
                        minutes_played = match_time - int(event_minute)
                        break
                
                performance_table[match_id][player_id]["position"] = player_position
                performance_table[match_id][player_id]["total_shots"] = re.sub('\t', '', total_shots)
                performance_table[match_id][player_id]["ot_shots"] = re.sub('\t', '', ot_shots)
                performance_table[match_id][player_id]["key_passes"] = re.sub('\t', '', key_passes)
                performance_table[match_id][player_id]["dribbles"] = re.sub('\t', '', dribbles)
                performance_table[match_id][player_id]["foul_given"] = re.sub('\t', '', foul_given)
                performance_table[match_id][player_id]["offsides"] = re.sub('\t', '', offsides)
                performance_table[match_id][player_id]["disp"] = re.sub('\t', '', disp)
                performance_table[match_id][player_id]["uns_touches"] = re.sub('\t', '', uns_touches)
                performance_table[match_id][player_id]["rating"] = rating
                performance_table[match_id][player_id]["key_events"] = key_events
                performance_table[match_id][player_id]["minutes_played"] = minutes_played

            #away_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #home_defensive_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-defensive\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_defensive_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-defensive\"]//tbody[@id=\"player-table-statistics-body\"]/tr")

            total_defensive_trs = home_defensive_trs + away_defensive_trs

            for tr in total_defensive_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_tackles = tr.find_element_by_class_name("TackleWonTotal ").text
                total_tackles = tr.find_element_by_xpath("td[4]").get_attribute("textContent")
                #interception = tr.find_element_by_class_name("InterceptionAll ").text
                interception = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #clearances = tr.find_element_by_class_name("ClearanceTotal ").text
                clearances = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #blocked_shots = tr.find_element_by_class_name("ShotBlocked ").text
                blocked_shots = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #fouls_committed = tr.find_element_by_class_name("FoulCommitted ").text
                fouls_committed = tr.find_element_by_xpath("td[8]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_tackles"] = re.sub('\t', '', total_tackles)
                performance_table[match_id][player_id]["interception"] = re.sub('\t', '', interception)
                performance_table[match_id][player_id]["clearances"] = re.sub('\t', '', clearances)
                performance_table[match_id][player_id]["blocked_shots"] = re.sub('\t', '', blocked_shots)
                performance_table[match_id][player_id]["fouls_committed"] = re.sub('\t', '', fouls_committed)

            #home_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-home-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            home_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-home-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            #away_passing_trs = subdriver.find_element_by_xpath("//*[@id=\"statistics-table-away-passing\"]").find_elements_by_xpath("//tbody[@id=\"player-table-statistics-body\"]/tr")
            away_passing_trs = subdriver.find_elements_by_xpath("//*[@id=\"statistics-table-away-passing\"]//tbody[@id=\"player-table-statistics-body\"]/tr")
            total_passing_trs = home_passing_trs + away_passing_trs

            for tr in total_passing_trs:
                player_id = int(re.findall(r'\d+', tr.find_element_by_class_name("player-link").get_attribute("href"))[0])

                #total_passes = tr.find_element_by_class_name("TotalPasses ").text
                total_passes = tr.find_element_by_xpath("td[5]").get_attribute("textContent")
                #pass_acc = tr.find_element_by_class_name("PassSuccessInMatch ").text
                pass_acc = tr.find_element_by_xpath("td[6]").get_attribute("textContent")
                #crosses = tr.find_element_by_class_name("PassCrossTotal ").text
                crosses = tr.find_element_by_xpath("td[7]").get_attribute("textContent")
                #acc_crosses = tr.find_element_by_class_name("PassCrossAccurate ").text
                acc_crosses = tr.find_element_by_xpath("td[8]").get_attribute("textContent")
                #total_long_balls = tr.find_element_by_class_name("PassLongBallTotal ").text
                total_long_balls = tr.find_element_by_xpath("td[9]").get_attribute("textContent")
                #acc_long_balls = tr.find_element_by_class_name("PassLongBallAccurate ").text
                acc_long_balls = tr.find_element_by_xpath("td[10]").get_attribute("textContent")
                #total_through_balls = tr.find_element_by_class_name("PassThroughBallTotal ").text
                total_through_balls = tr.find_element_by_xpath("td[11]").get_attribute("textContent")
                #acc_through_balls = tr.find_element_by_class_name("PassThroughBallAccurate ").text
                acc_through_balls = tr.find_element_by_xpath("td[12]").get_attribute("textContent")

                performance_table[match_id][player_id]["total_passes"] = re.sub('\t', '', total_passes)
                performance_table[match_id][player_id]["pass_acc"] = re.sub('\t', '', pass_acc)
                performance_table[match_id][player_id]["crosses"] = re.sub('\t', '', crosses)
                performance_table[match_id][player_id]["acc_crosses"] = re.sub('\t', '', acc_crosses)
                performance_table[match_id][player_id]["total_long_balls"] = re.sub('\t', '', total_long_balls)
                performance_table[match_id][player_id]["acc_long_balls"] = re.sub('\t', '', acc_long_balls)
                performance_table[match_id][player_id]["total_through_balls"] = re.sub('\t', '', total_through_balls)
                performance_table[match_id][player_id]["acc_through_balls"] = re.sub('\t', '', acc_through_balls)
                if (len(performance_table[match_id][player_id]) < 24):
                    raise
            break
        except:
            print(">>> error...waiting next trial...")
            time.sleep(1)
    
print(">>> finished.")
driver.close()
subdriver.close()

Total number of matches:380.


HBox(children=(IntProgress(value=0, max=380), HTML(value='')))

>>> 1: Crawling 1115400...



Exception in thread Thread-6:
Traceback (most recent call last):
  File "C:\Users\wcwong\AppData\Local\Continuum\Anaconda2\envs\tensorflow\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\wcwong\AppData\Local\Continuum\Anaconda2\envs\tensorflow\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\wcwong\AppData\Local\Continuum\Anaconda2\envs\tensorflow\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



>>> 2: Crawling 1115217...
>>> 3: Crawling 1115224...
>>> error...waiting next trial...
>>> error...waiting next trial...
>>> 4: Crawling 1115374...
>>> 5: Crawling 1115368...
>>> 6: Crawling 1115304...
>>> 7: Crawling 1115176...
>>> 8: Crawling 1115452...
>>> 9: Crawling 1115464...
>>> 10: Crawling 1115475...
>>> 11: Crawling 1115472...


In [15]:
import json

with open('../Data/{}-{}/performance_instances_table.json'.format(from_year,to_year), 'w') as fp:
    json.dump(performance_table, fp, indent=4)

In [188]:
import json

with open('./2017-2018/performance_instances_table.json', 'r') as fp:
    data = json.load(fp)

In [14]:
for match, match_info in performance_table.items():
    for player, player_info in match_info.items():
        if (len(player_info)< 24):
            print("match:{},player:{},len:{}".format(match,player,len(player_info)))

In [208]:
data['1225362']['106275']

KeyError: '106275'